In [1]:
# 创建pkl文件，用于后方索引
import numpy as np
import os
import pickle

data = dict()
path = './kapianfenlei/'
foldernames = os.listdir(path)
for foldername in foldernames:
    if not foldername.isdigit():
        continue
    filenames = os.listdir(path + foldername)
    for filename in filenames:
        data[filename] = foldername

pickle.dump(data,open('data_kapian.pkl','wb'))

In [2]:
# 修改总类别数
# 修改总类别数

In [1]:
# 修改总类别数
num_classes = 2



import pickle
import cv2
import numpy as np
import keras
from keras.applications.imagenet_utils import preprocess_input    # 改变编码
from keras.models import Model
from keras.preprocessing import image                             # 图像预处理
import matplotlib.pyplot as plt
import numpy as np
import pickle                                                     # 持久化
from random import shuffle


# 基本参数
size_jx = 100
batch_size = 128
input_shape = (size_jx, size_jx, 3)

gt = pickle.load(open('data_kapian.pkl', 'rb'))

keys = sorted(gt.keys())
num_train = int(round(0.9 * len(keys)))
train_keys = np.random.choice(keys, num_train, replace=False)
val_keys = set(keys) - set(train_keys)
val_keys = list(val_keys)
train_keys = keys
if num_classes < 10:
    val_keys = keys

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [2]:
class Generator(object):
    def __init__(self, gt,
                 batch_size, path_prefix,
                 train_keys, val_keys, image_size, leibieshu,
                 saturation_var=0.5,
                 brightness_var=0.5,
                 contrast_var=0.5,
                 lighting_std=0.5):
        self.gt = gt
        self.batch_size = batch_size
        self.path_prefix = path_prefix
        self.train_keys = train_keys
        self.val_keys = val_keys
        self.leibieshu = leibieshu
        self.train_batches = len(train_keys) * 2/ self.batch_size
        self.val_batches = len(val_keys) * 2/ self.batch_size
        self.image_size = image_size
        self.color_jitter = []
        if saturation_var:
            self.saturation_var = saturation_var
            self.color_jitter.append(self.saturation)
        if brightness_var:
            self.brightness_var = brightness_var
            self.color_jitter.append(self.brightness)
        if contrast_var:
            self.contrast_var = contrast_var
            self.color_jitter.append(self.contrast)
        self.lighting_std = lighting_std

    
    def grayscale(self, rgb):
        return rgb.dot([0.299, 0.587, 0.114])

    def saturation(self, rgb):
        gs = self.grayscale(rgb)
        alpha = 2 * np.random.random() * self.saturation_var 
        alpha += 1 - self.saturation_var
        rgb = rgb * alpha + (1 - alpha) * gs[:, :, None]
        return np.clip(rgb, 0, 255)

    def brightness(self, rgb):
        alpha = 2 * np.random.random() * self.brightness_var 
        alpha += 1 - self.saturation_var
        rgb = rgb * alpha
        return np.clip(rgb, 0, 255)

    def contrast(self, rgb):
        gs = self.grayscale(rgb).mean() * np.ones_like(rgb)
        alpha = 2 * np.random.random() * self.contrast_var 
        alpha += 1 - self.contrast_var
        rgb = rgb * alpha + (1 - alpha) * gs
        return np.clip(rgb, 0, 255)

    def lighting(self, img):
        cov = np.cov(img.reshape(-1, 3) / 255.0, rowvar=False)
        eigval, eigvec = np.linalg.eigh(cov)
        noise = np.random.randn(3) * self.lighting_std
        noise = eigvec.dot(eigval * noise) * 255
        img += noise
        return np.clip(img, 0, 255)
    

    def generate(self, train=True):
        while True:
            if train:
                shuffle(self.train_keys)
                keys = self.train_keys
            else:
                shuffle(self.val_keys)
                keys = self.val_keys
            inputs = []
            targets = []
            for key in keys:            
                img_path = self.path_prefix + self.gt[key] + '/' + key
                img = cv2.imread(img_path).astype('float32')
#                 if (str(self.gt[key]) != '0') & (np.random.random() < 0.5):
#                     img1= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#                     cv2.imwrite("./tmp.png", img1)
#                     img = cv2.imread("./tmp.png").astype('float32')
                y = self.gt[key]
                
#                 img = cv2.resize(img, self.image_size).astype('float32')
#                 img *= 255
                if train:
                    shuffle(self.color_jitter)
                    for jitter in self.color_jitter:
                        img = jitter(img)
                    if self.lighting_std:
                        img = self.lighting(img)

                targets.append(y)
                inputs.append(img)
                if len(targets) == self.batch_size:
                    tmp_inp = np.array(inputs, dtype=np.float64) / 255.
                    tmp_targets = np.array(targets)
                    tmp_targets = keras.utils.to_categorical(tmp_targets, self.leibieshu)
                    
                    inputs = []
                    targets = []
#                     print(tmp_targets)
                    yield tmp_inp, tmp_targets

In [3]:
# from net01 import fanpainet
# model = fanpainet(input_shape)
# 基本参数
# size_jx = 160
# batch_size = 128
# input_shape = (size_jx, size_jx, 3)
# num_classes = 4

# 网络部分
from keras.layers import Dense
from keras.layers import MaxPooling2D
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.models import Model

net = {}

net['input'] = Input(shape=input_shape)
# Block1
net['conv1'] = Conv2D(32, 3,
                      activation='relu',
                      padding='same',
                      name='conv1')(net['input'])
net['pool1'] = MaxPooling2D(4, 4, padding='same',
                            name='pool1')(net['conv1'])

# Block2
net['conv2'] = Conv2D(128, 3,
                      activation='relu',
                      padding='same',
                      name='conv2')(net['pool1'])
net['pool2'] = MaxPooling2D(4, 4, padding='same',
                            name='pool2')(net['conv2'])

# Block3
net['conv3'] = Conv2D(256, 3,
                      activation='relu',
                      padding='same',
                      name='conv3')(net['pool2'])
net['pool3'] = MaxPooling2D(3, 3, padding='same',
                            name='pool3')(net['conv3'])

# fc层
net['pool3_drop'] = Dropout(0.5, name='pool3_drop')(net['pool3'])
net['pool3_drop_fla'] = Flatten(name='pool3_drop_fla')(net['pool3_drop'])
net['fc1'] = Dense(512, activation='relu', name='fc1')(net['pool3_drop_fla'])

net['fc1_drop'] = Dropout(0.25, name='fc1_drop')(net['fc1'])
net['fc2'] = Dense(512, activation='relu', name='fc2')(net['fc1_drop'])

net['out'] = Dense(num_classes, activation='softmax', name='out')(net['fc2'])

# 创建model
model = Model(net['input'], net['out'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
path_prefix = './kapianfenlei/'

gen = Generator(gt, batch_size, path_prefix,
                train_keys, val_keys,
                (input_shape[0], input_shape[1]), num_classes)

In [5]:
def schedule(epoch, decay=0.9):
    return base_lr * decay**(epoch)

# # callbacks = [keras.callbacks.ModelCheckpoint('./checkpoints_weights/weights5.{epoch:02d}-{val_loss:.2f}.hdf5',
# #                                              verbose=1,
# #                                              save_weights_only=True),
# #              keras.callbacks.LearningRateScheduler(schedule)]
callbacks = [keras.callbacks.ModelCheckpoint('./checkpoints_weights/kapian.{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5',
                                             verbose=1,
                                             save_weights_only=True),
             keras.callbacks.LearningRateScheduler(schedule)]

In [6]:
# 进行编译
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.adam(),
#               metrics=['accuracy'])

# base_lr = 3e-4
# opt = keras.optimizers.Adam(lr=base_lr)
base_lr = 0.001
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=5e-04)

model.compile(optimizer=opt,
              loss=keras.losses.categorical_crossentropy,
              metrics = ['accuracy'])

In [7]:
# model.fit(train_x, train_y, batch_size, epochs=50, validation_data=(val_x,val_y))
nb_epoch = 10
history = model.fit_generator(gen.generate(True), gen.train_batches,
                              nb_epoch, verbose=1,
                              callbacks=callbacks,
                              validation_data=gen.generate(False),
                              validation_steps=gen.val_batches)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
9/8 [================================] - 10s 1s/step - loss: 0.3626 - acc: 0.8012 - val_loss: 0.0479 - val_acc: 1.0000

Epoch 00001: saving model to ./checkpoints_weights/kapian.01-0.36-0.80-0.05-1.00.hdf5
Epoch 2/10
9/8 [================================] - 9s 964ms/step - loss: 0.0822 - acc: 0.9878 - val_loss: 0.0449 - val_acc: 1.0000

Epoch 00002: saving model to ./checkpoints_weights/kapian.02-0.08-0.99-0.04-1.00.hdf5
Epoch 3/10
9/8 [================================] - 9s 1s/step - loss: 0.0813 - acc: 0.9974 - val_loss: 0.0432 - val_acc: 1.0000

Epoch 00003: saving model to ./checkpoints_weights/kapian.03-0.08-1.00-0.04-1.00.hdf5
Epoch 4/10
9/8 [================================] - 9s 989ms/step - loss: 0.0925 - acc: 0.9844 - val_loss: 0.0450 - val_acc: 1.0000

Epoch 00004: saving model to ./checkpoints_weights/kapian.04-0.09-0.98-0.04-1.00.hdf5
Epoch 5/10
9/8 [===================

In [8]:
base_lr = 0.001
opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=5e-04)

model.compile(optimizer=opt,
              loss=keras.losses.categorical_crossentropy,
              metrics = ['accuracy'])
# model.fit(train_x, train_y, batch_size, epochs=50, validation_data=(val_x,val_y))
nb_epoch = 10
history = model.fit_generator(gen.generate(True), gen.train_batches,
                              nb_epoch, verbose=1,
                              callbacks=callbacks,
                              validation_data=gen.generate(False),
                              validation_steps=gen.val_batches)

Epoch 1/10
9/8 [================================] - 10s 1s/step - loss: 0.1211 - acc: 0.9635 - val_loss: 2.3724e-04 - val_acc: 1.0000

Epoch 00001: saving model to ./checkpoints_weights/kapian.01-0.12-0.96-0.00-1.00.hdf5
Epoch 2/10
9/8 [================================] - 9s 997ms/step - loss: 0.0141 - acc: 0.9974 - val_loss: 2.3017e-04 - val_acc: 1.0000

Epoch 00002: saving model to ./checkpoints_weights/kapian.02-0.01-1.00-0.00-1.00.hdf5
Epoch 3/10
9/8 [================================] - 9s 995ms/step - loss: 0.0143 - acc: 0.9983 - val_loss: 2.4657e-04 - val_acc: 1.0000

Epoch 00003: saving model to ./checkpoints_weights/kapian.03-0.01-1.00-0.00-1.00.hdf5
Epoch 4/10
9/8 [================================] - 9s 973ms/step - loss: 0.0201 - acc: 0.9931 - val_loss: 2.3658e-04 - val_acc: 1.0000

Epoch 00004: saving model to ./checkpoints_weights/kapian.04-0.02-0.99-0.00-1.00.hdf5
Epoch 5/10
9/8 [================================] - 9s 1s/step - loss: 0.0195 - acc: 0.9948 - val_loss: 2.3097

In [ ]:
base_lr = 0.001
opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=5e-04)

model.compile(optimizer=opt,
              loss=keras.losses.categorical_crossentropy,
              metrics = ['accuracy'])
# model.fit(train_x, train_y, batch_size, epochs=50, validation_data=(val_x,val_y))
nb_epoch = 10
history = model.fit_generator(gen.generate(True), gen.train_batches,
                              nb_epoch, verbose=1,
                              callbacks=callbacks,
                              validation_data=gen.generate(False),
                              validation_steps=gen.val_batches)

In [9]:
model.save_weights('kapian4.h5')

In [1]:
# 进入kapianfenlei.py修改内容，在第一行
'''
    接口说明：和卡片识别接口函数完全一致，具体可看卡片识别说明
    其中仅调用工具包的位置不同，卡片识别从dazuiniao调用，此处从kapianfenlei调用，调用工具包代码如下
    from kapianfenlei import kapian_out
    out, img, _ = kapian_out(frame, 7, 15)
'''
from kapianfenlei import kapian_out
import cv2
cap = cv2.VideoCapture(0)

while(True):
    ref, frame=cap.read()
    out, img, _ = kapian_out(frame, 7, 15)
    cv2.imshow('out', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\f4106\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\f4106\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\f4106\AppData\Roaming\Pyth

C:\Users\f4106\AppData\Roaming\Python\Python36\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\f4106\AppData\Roaming\Python\Python36\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\f4106\AppData\Roaming\Python\Python36\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\f4106\AppData\Roaming\





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

